In [1]:
# importing python modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scanpy as sc

%matplotlib inline
sc.logging.print_header()

scanpy==1.8.1 anndata==0.7.5 umap==0.5.1 numpy==1.21.0 scipy==1.6.2 pandas==1.2.3 scikit-learn==0.23.2 statsmodels==0.12.2 python-igraph==0.9.1 louvain==0.7.0 leidenalg==0.8.2 pynndescent==0.5.2


In [2]:
adata = sc.read_h5ad('SS2_processed.h5ad')

In [3]:
adata

AnnData object with n_obs × n_vars = 2090 × 34041
    obs: 'plate_ID', 'well', 'cell_id', 'Tumor_ID', 'Animal_ID', 'receptor_status', 'ER_receptor', 'intrinsic_BC_type', 'metastatic_potential_rank', 'metastatic_potential_group', 'sort', 'n_counts', 'n_genes', 'percent_ERCC', 'percent_mito', 'percent_ribo', 'sequencing_batch', 'S_score', 'G2M_score', 'phase', 'louvain', 'leiden', 'E_score', 'M_score', 'EMP_score', 'EMP_stage'
    var: 'n_cells', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'mean', 'std'
    uns: 'EMP_stage_colors', 'ER_receptor_colors', 'Tumor_ID_colors', 'hvg', 'intrinsic_BC_type_colors', 'leiden', 'leiden_colors', 'louvain', 'louvain_colors', 'neighbors', 'pca', 'ranking_ID_colors', 'receptor_status_colors', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'log1p', 'raw_counts'
    obsp: 'connectivities', 'distances'

In [4]:
set(adata.obs['sort'])

{'Metastatic', 'Tumor'}

In [5]:
adata_tumor = adata[adata.obs['sort'] == 'Tumor']
adata_tumor

/home/weilun/anaconda3/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


View of AnnData object with n_obs × n_vars = 1395 × 34041
    obs: 'plate_ID', 'well', 'cell_id', 'Tumor_ID', 'Animal_ID', 'receptor_status', 'ER_receptor', 'intrinsic_BC_type', 'metastatic_potential_rank', 'metastatic_potential_group', 'sort', 'n_counts', 'n_genes', 'percent_ERCC', 'percent_mito', 'percent_ribo', 'sequencing_batch', 'S_score', 'G2M_score', 'phase', 'louvain', 'leiden', 'E_score', 'M_score', 'EMP_score', 'EMP_stage'
    var: 'n_cells', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'mean', 'std'
    uns: 'EMP_stage_colors', 'ER_receptor_colors', 'Tumor_ID_colors', 'hvg', 'intrinsic_BC_type_colors', 'leiden', 'leiden_colors', 'louvain', 'louvain_colors', 'neighbors', 'pca', 'ranking_ID_colors', 'receptor_status_colors', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'log1p', 'raw_counts'
    obsp: 'connectivities', 'distances'

In [8]:
bash_file = open("SS2_1_vs_rest.sh", "w")
for group in set(adata_tumor.obs['metastatic_potential_group']):
    sub_adata_group = adata_tumor[adata_tumor.obs['metastatic_potential_group'] == group].copy()
    for tumor in set(sub_adata_group.obs['Tumor_ID']):
        sub_adata_tumors = sub_adata_group[~sub_adata_group.obs['Tumor_ID'].isin([tumor])].copy()
        final_sub = adata_tumor[~adata_tumor.obs.index.isin(sub_adata_tumors.obs.index)].copy()
        final_sub.write_h5ad('SS2_'+tumor+'_vs_rest.h5ad')
        bash_file.write('Rscript single_tumor_vs_other_groups.R SS2_'+tumor+'_vs_rest.h5ad SS2_'+tumor+'_vs_rest.h5seurat '+tumor+' '+group)
        bash_file.write("\n")